This notebook contains code to run simulations that produced the ../results/data/*.pkl files.
These files are then loaded in ../archive/report_calculations_and_plots.ipynb to produce the figures and tables in the report.
It is intendended to run on Google Colab, hence repo is cloned. 

In [ ]:
!git clone https://github.com/Leonard-P/orbitronics_research_project.git orb
%cd orb

# This version was used for the files in ../results/data/
!git checkout ae3b63e502d485679a08cc784d3232af19cf6fd6

In [ ]:
import sys
import tqdm.notebook as tqnb

# overwrite the plain tqdm with the notebook-friendly one
sys.modules["tqdm"].tqdm = tqnb.tqdm

from lattice import *
from lattice.lattice_rk4_gpu import OrbitalPolarisationWithShapeGPU, DynamicsFrameRecorderGPU
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from google.colab import files
import shutil

In [ ]:
def triple_fold_simulation(Lx, Ly, omega, decay_time, data_folder):
    T = 2 * np.pi / omega
    cycles = 17
    T_ramp = 3 * 2 * np.pi / 0.5
    field_gen = FieldAmplitudeGenerator.ramped_oscillation(amplitude=1e-3, omega=omega, ramp_width=T_ramp)

    geometry = HexagonalLatticeGeometry((Lx, Ly))
    edges = np.array(geometry.edges)


    if not os.path.isdir(data_folder):
        os.makedirs(data_folder)

    for i in range(3):
        # perform 3 simulations, double the time resolution for each
        substeps = 2 * 25 * 2**i

        params = SimulationParameters(
            t_hop=-1.0,
            E_amplitude=field_gen,
            E_direction=np.array([0, 1]),
            h=2.0/omega,
            T=cycles*T + T_ramp,
            substeps=substeps,
            initial_occupation=0.5
        )


        l = Lattice2D(geometry, params)

        orb_pol_obs = OrbitalPolarisationWithShapeGPU(
            site_positions=np.array([geometry.site_to_position(i) for i in range(l.N)]),
            curl_origin=geometry.curl_origin,
            cell_anchor_sites=np.array(geometry.get_curl_sites()),
            cell_path_offsets=np.array([(0, 1), (1, 2), (2, l.Lx + 2), (l.Lx + 2, l.Lx + 1), (l.Lx + 1, l.Lx), (l.Lx, 0)]),
            t_hop=params.t_hop,
            m=0.741, # electron mass in natural units
            a_nn=1.0,
            Area=len(geometry.get_curl_sites())*3*np.sqrt(3)/2,
            sample_every=substeps//25,
        )

        animation_obs = DynamicsFrameRecorderGPU(
            nn_rows=edges[:, 0],
            nn_cols=edges[:, 1],
            site_positions=np.array([geometry.site_to_position(i) for i in range(l.N)]),
            cell_anchor_sites=np.array(geometry.get_curl_sites()),
            cell_path_offsets=np.array([(0, 1), (1, 2), (2, l.Lx + 2), (l.Lx + 2, l.Lx + 1), (l.Lx + 1, l.Lx), (l.Lx, 0)]),
            start_index=((cycles-1)*T + T_ramp) / (l.h/substeps),
            steps=T/(l.h/substeps),
            sample_every=int(T/(l.h/substeps) / 20), # 20 frames
            t_hop=params.t_hop,
            a_nn=1.0,
            m=0.741,
        )

        l.evolve(
            decay_time=decay_time,
            observables_gpu=[orb_pol_obs, animation_obs],
            use_gpu=True,
            precision="single",
        )

        del l

        t_vals = np.linspace(0, cycles*T + T_ramp, len(orb_pol_obs.values))
        E_vals = np.array([field_gen(t) for t in t_vals])

        dump = {
            "Lx": Lx,
            "Ly": Ly,
            "omega": omega,
            "decay_time": decay_time,
            "h": params.h,
            "substeps": substeps,
            "E_time": t_vals,
            "E_amplitude": E_vals,
            "orb_pol_values": orb_pol_obs.values,
            "animation_values": animation_obs.values,
        }

        fname = f"Lx{Lx}_Ly{Ly}_omega{omega:.3f}_sub{substeps}.pkl"
        path = os.path.join(data_folder, fname)

        with open(path, "wb") as f:
            pickle.dump(dump, f)

In [ ]:
import time
start = time.time()

#Lx_range = [11, 19, 30]
Lx_range = [29]
Ly_range = [20, 40, 80]
omega_range = [0.5, 1.0, 2.0]

for Lx in Lx_range:
    for Ly in Ly_range:
        for omega in omega_range:
            print(f"Lx: {Lx}, Ly: {Ly}, omega: {omega}")
            triple_fold_simulation(Lx, Ly, omega, 14.0, "../data_final")

print("Parameter grid done, resimulating without decay.")
triple_fold_simulation(19, 40, 1.0, float("inf"), "../data_nodecay_final")

print("Done.")
print(f"Simulation took {(time.time() - start) / 60: .2f} minutes.")


outpath = "../data_final"
shutil.make_archive(outpath, "zip", "../data_final")
shutil.make_archive(outpath + "_nodecay", "zip", "../data_nodecay_final")

files.download(outpath + ".zip")
files.download(outpath + "_nodecay.zip")